In [25]:
import yfinance as yf
import pandas as pd 
import numpy as np
import ta
import datetime as dt

In [26]:
df = yf.download('TQQQ', start='2022-01-15',interval='30m')

[*********************100%***********************]  1 of 1 completed


In [27]:
def manipulations(df):
    df['EMA200'] = ta.trend.ema_indicator(df.Close,window=200)
    df['wf_Top_bool'] = np.where(
    df['High']==df['High'].rolling(9,center=True).max(),True,False)
    df['wf_top'] = np.where(
    df['High']==df['High'].rolling(9,center=True).max(),df['High'],None
    )
    df['wf_top'] = df['wf_top'].ffill()
    df.dropna(inplace=True)
    df['Buy'] = np.where((df.Close> df.wf_top) & (df.Close>df.EMA200),1,0)
    df['SL'] = np.where(df.Buy==1, df.Close-(df.Close-df.Low),0)
    df['TP'] = np.where(df.Buy==1, df.Close+(df.Close-df.Low)*1.5,0)
    return df
def profits(df):
    buys, sells = [],[]
    for i in range(len(df)):
        if df.Buy.iloc[i]:
            buys.append(df.iloc[i].name)
            for j in range(len(df)-i):
                if df.TP.iloc[i] < df.Close.iloc[i+j] or \
                df.SL.iloc[i] > df.Close.iloc[i+j]:
                    sells.append(df.iloc[i+j].name)
                    break
    frame = pd.DataFrame([buys,sells]).T.dropna()
    frame.columns = ['Buys','Sells']
    actualtrades = frame[frame.Buys>frame.Sells.shift(1)]
    profits = (df.loc[actualtrades.Sells].Close.values - \
    df.loc[actualtrades.Buys].Close.values) / df.loc[actualtrades.Buys].Close.values
    return profits
def check_strat(stonks):
    res = {'ticker':[],'final_profit':[],'pcnt_change':[]}
    dates = (dt.datetime.now() - dt.timedelta(30)).strftime('%Y-%m-%d')
    for i in stonks:
        res['ticker'].append(i)
        df = yf.download(i,start=dates,interval='15m')
        base = df.copy()
        df = manipulations(df)
        profit = profits(df)
        res['final_profit'].append(sum(profit))
        res['pcnt_change'].append(((base.Close.pct_change() +1).cumprod()[-1]))
    df=pd.DataFrame(res)
    return df       

In [54]:
final = check_strat(['RTX'])

[*********************100%***********************]  1 of 1 completed


In [55]:
final_copy = final.copy()
final_copy['pcnt_change'] = final_copy['pcnt_change'] -1
final_copy['win_delta'] = final_copy['final_profit'] - final_copy['pcnt_change']
final_copy.set_index('ticker', inplace=True)
final_copy.sort_values('win_delta', ascending=False)
final_copy

,final_profit,pcnt_change,win_delta
ticker,,,
RTX,0.038982,0.015826,0.023156
